有 n 位乘客即将登机，飞机正好有 n 个座位。第一位乘客的票丢了，他随便选了一个座位坐下。
剩下的乘客将会：如果他们自己的座位还空着，就坐到自己的座位上， 当他们自己的座位被占用时，随机选择其他座位，第 n 位乘客坐在自己的座位上的概率是多少？

示例 1：输入：n = 1 输出：1.00000 解释：第一个人只会坐在自己的位置上。

示例 2：输入: n = 2 输出: 0.50000 解释：在第一个人选好座位坐下后，第二个人坐在自己的座位上的概率是 0.5。

提示：1 <= n <= 10^5
用random 模拟求解

解题思路[1]
对于第一个乘客来说 他有三种选择：

坐在正确的（自己的位置）, 那么后面的乘客都不会乱，所以第n个乘客可以坐到自己的位置, 1/n * 1.
坐在第n个乘客的位置，那么第n个乘客肯定无法坐到自己的位置, 1/n * 0.
坐在[1,n-1]之间的某个位置K.
对于第K个乘客而言，自己的位置K已经被乘客1给占了，而[2，K-1]的乘客先于K乘客 上飞机，能找到自己的位置并坐下，所以当K乘客上飞机时，留给他的选择是

第1个座位，以及[K+1,n]的座位。

此时K乘客同样有3个选择：

如果他坐在正确的座位，那么后面的乘客都不会乱，第n个乘客可以坐到自己的位置，只不过此时对于K乘客而言，正确的座位就是座位1。
坐在第n个乘客的位置，那么第n个乘客肯定无法坐到自己的位置
坐在[K+1,n-1]之间的某个位置。
可以发现对于第一个乘客和第K个乘客，他们面临的选择是一样的，只不过问题的规模不一样。第K个乘客时，问题的规模只有n-K+1. (为何， 上面已经解释过了，对于第K个乘客而言，自己的位置K已经被乘客1给占了，而[2，K-1]的乘客先于K乘客 上飞机，能找到自己的位置并坐下)。

所以此题公式为：

f(1)=1

f(2)=0.5

f(3)=1/3 + 1/3 * 1/2  = 0.5

f(4)=1/4 + 1/4*(1/3+ 1/3 * 1/2  ) + 1/4* 1/2 = 1/4 + 1/4 * f(3) + 1/4* f(2)=0.5

f(5)=1/5 + 1/5 * f(4) + 1/5 * f(3) + 1/5 * f(2)=0.5

...

f(n)=1/n + 1/n * f(n-1) + 1/5 * f(n-2) + ... + 1/n * f(2) =0.5
 

In [63]:
#暴力模拟法
import random
def simulate_bruteforce(n):    
    """    Simulates one round. Unbounded time complexity.    
    :param n: total number of seats    
    :return: True if last one has last seat, otherwise False    
    """    
    seats = [False for _ in range(n)]    
    for i in range(n-1):       
        if i == 0:  # first one, always random            
            seats[random.randint(0, n - 1)] = True        
        else:            
            if not seats[i]:  # i-th has his seat                
                seats[i] = True            
            else:                
                while True:                    
                    rnd = random.randint(0, n - 1) # random until no conflicts                    
                    if not seats[rnd]:                        
                        seats[rnd] = True                        
                        break    
    return not seats[n-1]
ans = [simulate_bruteforce(i) for i in range(2,1000) ] *1000
sum(ans )/len(ans)
    

0.49899799599198397

In [58]:
#标准答案
class Solution:    
    def nthPersonGetsNthSeat(self, n: int) -> float:        
        return 1.0 if n == 1 else 0.5

ans = Solution().nthPersonGetsNthSeat(50)
ans

0.5

In [64]:
def simulate_online(n: int) -> bool:    
    """    Simulates one round of complexity O(N).    
    :param n: total number of seats    
    :return: True if last one has last seat, otherwise False    
    """    
    seats = [i for i in range(n)]    
    def swap(i, j):        
        tmp = seats[i]        
        seats[i] = seats[j]        
        seats[j] = tmp    
    # for each person, the seats array idx available are [i, n-1]    
    for i in range(n-1):        
        if i == 0:  # first one, always random            
            rnd = random.randint(0, n - 1)            
            swap(rnd, 0)        
        else:            
            if seats[i] == i:  # i-th still has his seat                
                pass           
            else:                
                rnd = random.randint(i, n - 1)  # selects idx from [i, n-1]                
                swap(rnd, i)    
    return seats[n-1] == n - 1
            
ans = [simulate_online(i) for i in range(2,10000) ] *1000
sum(ans )/len(ans)

0.5033006601320265

In [65]:
def simulate(n: int) -> bool:    
    """    Simulates one round of complexity O(N).    
    :param n: total number of seats    
    :return: True if last one has last seat, otherwise False    
    """    
    seats = [False for _ in range(n)]    
    # for each person, the seats array idx available are [i, n-1]    
    for i in range(n-1):        
        if i == 0:  # first one, always random            
            rnd = random.randint(0, n - 1)            
            seats[rnd] = True        
        else:            
            if not seats[i]:  # i-th still has his seat                
                seats[i] = True            
            else:                # 0 must not be available, now we have 0 and [i+1, n-1],                
                rnd = random.randint(i, n - 1)                
                if rnd == i:                    
                    seats[0] = True                
                else:                    
                    seats[rnd] = True    
    return not seats[n-1]

ans = [simulate(i) for i in range(2,10000) ] *1000
sum(ans )/len(ans)

0.503500700140028

In [ ]:
# 罗毅扬 best
import random
n = 100#人数
k = 100000#次数
sum = 0#统计次数
for i in range(k):
    x = 1#表明第一个乘客开始
    while True:
        x = random.randint(x+1,n+1)
        if x == n+1:
            sum+=1
            break
        elif x == n:
            break
        else:
            None
print(sum/k)#输出结果